This part contains the following steps:
1. Reduction du nombre de couleur
2. Faire l'histogramme de couleur d'une image
3. Compute the distance matrix between each bubble, using its color histogram.

# Notebook de Temano

Ici, Temano peut faire ses tests pour les parties du sujet dont il s'occupe.

Bon courage Temano :)

In [25]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import os

In [26]:
# load the base image and its mask.
poster = cv2.imread(os.path.join("ressources/train/03/", "c03_s04.png"))
# let us convert the poster from BGR to RGB for what follows
# we will work with RGB images
poster = cv2.cvtColor(poster, cv2.COLOR_BGR2RGB)
poster.shape, poster.dtype

((211, 212, 3), dtype('uint8'))

In [27]:
plt.subplot(1,2,1)
plt.imshow(poster)  # no need for `cv2.cvtColor` because we already have an RGB image

In [28]:
# TODO now sample 5000 pixels from the image
sample_pixels = np.array(poster, 5000)  # FIXME

sample_pixels.shape, sample_pixels.dtype

TypeError: Cannot interpret '5000' as a data type